<a href="https://colab.research.google.com/github/dauphin95/data_spring/blob/master/StandardScaler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np                   # array, vector, matrix calculations
import pandas as pd                  # DataFrame handling
from sklearn.model_selection import train_test_split
import xgboost as xgb                # gradient boosting machines (GBMs)
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('credit_cards_dataset.csv')
X = df.drop(['ID', 'default.payment.next.month'], axis=1).values
Y = df['default.payment.next.month'].values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

## 평균 0, 분산 1 -> 같은 scaler로 바뀜
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.fit_transform(X_test)

In [ ]:
# XGBoost Regressor
mod = xgb.XGBRegressor(
    gamma=1,                 
    learning_rate=0.01,
    max_depth=3,
    n_estimators=10000,                                                                    
    subsample=0.8,
    random_state=42,
    verbosity=1 )
mod.fit(X_train_sc, Y_train)

## 예측
ypred = mod.predict(X_test_sc)

## 회귀 분석 L1, L2 에러값 출력
import math
from sklearn.metrics import mean_squared_error
rmse = math.sqrt(mean_squared_error(Y_test, ypred))
print(rmse)
mae = mean_squared_error(Y_test, ypred)
print(mae)


[05:07:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
## 분류 분석 에러값 출력
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import itertools

predictions = np.rint(ypred)
accuracy = accuracy_score(Y_test, predictions)
cm = confusion_matrix(Y_test, predictions)
precision = precision_score(Y_test, predictions)
recall = recall_score(Y_test, predictions)
print(f'accuracy = {accuracy}' )
print(f'cm = {cm}')
print(f'precision = {precision}')
print(f'recall = {recall}')

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
plt.figure()
plot_confusion_matrix(cm, classes=['Non_Default','Default'], normalize=False,
                      title='Non Normalized confusion matrix')